## Install and import bibraries

In [2]:
# !pip install -qq datasets==2.16.1 evaluate==0.4.1 transformers[sentencepiece]==4.35.2
# !pip install -qq accelerate==0.26.1
# !apt install git-lfs

In [2]:
import numpy as np
from tqdm.auto import tqdm
import collections

import torch

from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForQuestionAnswering
from transformers import TrainingArguments
from transformers import Trainer
import evaluate

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

d:\Asus\Ungdung\Miniconda\workspace\envs\AIOEx\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Sử dụng mô hình "distilbert-base-uncased"
# làm mô hình checkpoint
MODEL_NAME = "distilbert-base-uncased"

# Độ dài tối đa cho mỗi đoạn văn bản
# sau khi được xử lý
MAX_LENGTH = 384

# Khoảng cách giữa các điểm bắt đầu
# của các đoạn văn bản liên tiếp
STRIDE = 128

## Setup Dataset

### Load datasets

In [4]:
DATASET_NAME = "rajpurkar/squad_v2"
raw_datasets = load_dataset(DATASET_NAME)

In [5]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 130319
    })
    validation: Dataset({
        features: ['id', 'title', 'context', 'question', 'answers'],
        num_rows: 11873
    })
})

### Tokenizer

In [6]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [7]:
context = raw_datasets["train"][0]["context"]
question = raw_datasets["train"][0]["question"]

inputs = tokenizer(question, context)
tokenizer.decode(inputs["input_ids"])

'[CLS] when did beyonce start becoming popular? [SEP] beyonce giselle knowles - carter ( / biːˈjɒnseɪ / bee - yon - say ) ( born september 4, 1981 ) is an american singer, songwriter, record producer and actress. born and raised in houston, texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of r & b girl - group destiny \' s child. managed by her father, mathew knowles, the group became one of the world \' s best - selling girl groups of all time. their hiatus saw the release of beyonce \' s debut album, dangerously in love ( 2003 ), which established her as a solo artist worldwide, earned five grammy awards and featured the billboard hot 100 number - one singles " crazy in love " and " baby boy ". [SEP]'

In [83]:
raw_datasets["train"][0]

{'id': '56be85543aeaaa14008c9063',
 'title': 'Beyoncé',
 'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'question': 'When did Beyonce start becoming popular?',
 'answers': {'text': ['in the late 1990s'], 'answer_start': [269]}}

In [87]:
def find_start_end(inputs, examples, offset_mapping, sample_map):
    # Trích xuất thông tin về câu trả lời (answers) từ examples.
    answers = examples["answers"]

    # Khởi tạo danh sách các vị trí bắt đầu và kết thúc câu trả lời.
    start_positions = []
    end_positions = []

    # Duyệt qua danh sách offset_mapping.
    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]

        # Trích xuất sequence_ids.
        sequence_ids = inputs.sequence_ids(i)

        # xác định start-end id of context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        start_idx = idx
        while sequence_ids[idx] == 1:
            idx += 1
        end_idx = idx - 1

        # Trích xuất thông tin về câu trả lời cho mẫu này
        answer = answers[sample_idx]

        # Nếu không có câu trả lời, gán nhãn (0, 0).
        if len(answer['text']) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Xác định vị trí ký tự bắt đầu và kết thúc của câu trả lời trong ngữ cảnh.
            start_char = answer["answer_start"][0]
            end_char = answer["answer_start"][0] + len(answer["text"][0])

            # Nếu câu trả lời không nằm hoàn toàn trong ngữ cảnh, gán nhãn là (0, 0).
            if offset[start_idx][0] > start_char or offset[end_idx][1] < end_char:
                start_positions.append(0)
                end_positions.append(0)
            else:
                # Nếu không, xác định vị trí token bắt đầu và kết thúc của câu trả lời.
                idx = start_idx
                while idx <= end_idx and offset[idx][0] <= start_char:
                    idx += 1
                start_positions.append(idx - 1)

                idx = end_idx
                while idx >= start_idx and offset[idx][1] >= end_char:
                    idx -= 1
                end_positions.append(idx + 1)

    return start_positions, end_positions
        

In [88]:
# Định nghĩa hàm preprocess_training_examples và nhận đối số examples là dữ liệu đào tạo
def preprocess_training_examples(examples): # batch of samples
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length")
    # Trích xuất offset_mapping và sample_map, loại bỏ chúng khỏi inputs.
    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    
    start_positions, end_positions = find_start_end(
        inputs, examples, offset_mapping, sample_map)

    # Thêm thông tin vị trí bắt đầu và kết thúc vào inputs.
    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions

    return inputs

In [89]:
# Tạo một biến train_dataset và gán cho nó giá trị sau khi áp dụng hàm preprocess_training_examples lên tập dữ liệu "train"
# Bật chế độ xử lý theo từng batch bằng cách đặt batched=True
# Loại bỏ các cột không cần thiết trong tập dữ liệu "train" bằng cách sử dụng remove_columns

train_dataset = raw_datasets["train"].map(
    preprocess_training_examples,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
)

# In ra độ dài của tập dữ liệu "train" ban đầu và độ dài của tập dữ liệu đã được xử lý (train_dataset)
len(raw_datasets["train"]), len(train_dataset)

Map: 100%|██████████| 130319/130319 [00:38<00:00, 3370.76 examples/s]


(130319, 131754)

In [93]:
print(train_dataset[0].keys())

dict_keys(['input_ids', 'attention_mask', 'start_positions', 'end_positions'])


In [73]:
# Định nghĩa hàm preprocess_training_examples và nhận đối số examples là dữ liệu đào tạo
def preprocess_validation_examples(examples):  # batch of samples
    inputs = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=MAX_LENGTH,
        stride=STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length")
    # Lấy ánh xạ để ánh xạ lại ví dụ tham chiếu cho từng dòng trong inputs
    sample_map = inputs.pop("overflow_to_sample_mapping")
    example_ids = []

    # Xác định ví dụ tham chiếu cho mỗi dòng đầu vào và
    # điều chỉnh ánh xạ offset
    for i in range(len(inputs["input_ids"])):
        sample_idx = sample_map[i]
        example_ids.append(examples["id"][sample_idx])

        sequence_ids = inputs.sequence_ids(i)
        offset = inputs["offset_mapping"][i]

        # Loại bỏ các offset không phù hợp với sequence_ids (chỉ giữ lại offset của context)
        inputs["offset_mapping"][i] = [
            o if sequence_ids[k] == 1 else None
            for k, o in enumerate(offset)
        ]

    # Thêm thông tin ví dụ tham chiếu vào đầu vào
    inputs["example_id"] = example_ids

    return inputs

In [74]:
# Tạo một biến validation_dataset và gán giá trị bằng việc sử dụng dữ liệu từ raw_datasets["validation"] sau khi áp dụng một hàm xử lý trước.
validation_dataset = raw_datasets["validation"].map(
    # Gọi hàm preprocess_validation_examples để xử lý dữ liệu đầu vào.
    preprocess_validation_examples,
    batched=True,  # Xử lý dữ liệu theo từng batch.
    # Loại bỏ các cột không cần thiết từ dữ liệu ban đầu.
    remove_columns=raw_datasets["validation"].column_names,
)

# In ra độ dài của raw_datasets["validation"] và validation_dataset để so sánh.
len(raw_datasets["validation"]), len(validation_dataset)

Map: 100%|██████████| 11873/11873 [00:05<00:00, 2157.49 examples/s]


(11873, 12134)

In [94]:
validation_dataset[0].keys()

dict_keys(['input_ids', 'attention_mask', 'offset_mapping', 'example_id'])

## Train model

In [ ]:
# Load model
model = AutoModelForQuestionAnswering.from_pretrained(MODEL_NAME)

In [ ]:
training_args = TrainingArguments(
    output_dir="/kaggle/working/distilbert-finetuned-squadv2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=15,
    weight_decay=0.01,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    report_to=None,
    save_total_limit=1,
    metric_for_best_model="eval_accuracy"
)

# Khởi tạo một đối tượng Trainer để huấn luyện mô hình
trainer = Trainer(
    model=model,  # Sử dụng mô hình đã tạo trước đó
    args=training_args,  # Các tham số và cấu hình huấn luyện
    train_dataset=train_dataset,  # Sử dụng tập dữ liệu huấn luyện
    eval_dataset=validation_dataset,  # Sử dụng tập dữ liệu đánh giá
    tokenizer=tokenizer,  # Sử dụng tokenizer để xử lý văn bản
)

# Bắt đầu quá trình huấn luyện
trainer.train()

## Evaluate model